In [7]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(os.getcwd(), '..', '..'))

sys.path.append(os.path.join(os.getcwd(), '..', '..', '..', 'pplSIT', 'workflow', 'utils'))

In [11]:
%matplotlib inline

from session.sessions import selected_009266, selected_008229, selected_009265
from imports import *
from loading import load_session_data
from behavior import get_behav_units
from population import unit_response_matrix, activity_at_phase

from scipy import stats
from scipy import signal
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import decomposition
from sklearn.cluster import DBSCAN

from umap import UMAP  # conda install -c conda-forge umap-learn
from sklearn.manifold import TSNE

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [13]:
source = '/home/sobolev/nevermind_ag-grothe/AG_Pecka/data/processed/'
source = '/home/sobolev/nevermind/AG_Pecka/data/processed/'

sessions = [s for s in selected_009266.keys()]
#sessions = [s for s in selected_009265.keys()]
#sessions = [s for s in selected_008229.keys()]
sessions.sort()

try:
    sessions.remove('009266_hippoSIT_2023-04-20_15-24-14')
except:
    pass
try:
    sessions.remove('009265_hippoSIT_2023-02-27_10-18-32')
    sessions.remove('009265_hippoSIT_2023-02-27_15-33-46')
except:
    pass

# special for 009265
#sessions = sessions[7:-1]

selected = [
    '009265_hippoSIT_2023-03-09_20-03-08',
    '009265_hippoSIT_2023-03-05_11-52-17'
]
selected

['009265_hippoSIT_2023-03-09_20-03-08', '009265_hippoSIT_2023-03-05_11-52-17']

In [14]:
def get_ratio_matrix(moseq, tl, win_l=2, step=1, s_rate=100, syl_num=10):
    idxs_srm_tl = np.arange(0, len(tl), int(step*s_rate))
    syl_ratio_mx = np.zeros([len(idxs_srm_tl), syl_num])
    for k, idx in enumerate(idxs_srm_tl):
        curr_syls = moseq[:, 1][idx:idx + int(win_l*s_rate)]  # second column is syllables reindexed
        for j in np.arange(syl_num):
            syl_ratio_mx[k, j] = np.sum(curr_syls == j) / int(win_l*s_rate)

    # roll 1 step to match
    syl_ratio_mx = np.roll(syl_ratio_mx, 1, axis=0)
    syl_ratio_mx[0] = syl_ratio_mx[1]
    
    return syl_ratio_mx, idxs_srm_tl

## t-SNE / UMAP for first 10 syllables

In [21]:
umap_dists        = [0.1, 0.3, 0.5, 0.7]
perplexities      = [20, 50, 70, 100]
umap_fits         = {}
tsne_fits         = {}
idxs_srm_succ_all = {}
idxs_srm_fail_all = {}
win_l, step = 2, 0.25  # in seconds
    
for i, session in enumerate(selected):
    # load data
    animal = session.split('_')[0]
    s_path           = os.path.join(source, animal, session)
    moseq_file       = os.path.join(source, animal, session, 'MoSeq.h5')
    meta_file        = os.path.join(source, animal, session, 'meta.h5')
    moseq_class_file = os.path.join(source, animal, session, 'analysis', 'MoSeq_WX.h5')
    
    with h5py.File(moseq_file, 'r') as f:
        moseq = np.array(f['moseq'])
        headers = list(f['moseq'].attrs['headers'])

    with h5py.File(meta_file, 'r') as f:
        tl = np.array(f['processed']['timeline'])
        tgt_mx = np.array(f['processed']['target_matrix'])
        events = np.array(f['processed']['sound_events'])
        
    syl_ratio_mx, idxs_srm_tl = get_ratio_matrix(moseq, tl, win_l=win_l, step=step)
    
    w_mx = []
    for phase in [1, 2, 3, 4]:
        w_pca = activity_at_phase(s_path, phase, do_pca=True)
        w_mx.append(w_pca) # stay in events space
    w_mx = np.column_stack(w_mx)
    
    l_events = len(events)
    combined_mx = np.column_stack([syl_ratio_mx[:l_events], w_mx[:l_events]])

    # indices of syllable ratio matrix when animal was successfully sitting
    # in the island
    idxs_srm_succ = []
    for tgt_rec in tgt_mx[tgt_mx[:, 4] == 1]:
        tl_l, tl_r = tgt_rec[2], tgt_rec[3]
        idxs_srm_succ += list(np.where((idxs_srm_tl > tl_l) & (idxs_srm_tl < tl_r))[0])
    idxs_srm_succ_all[session] = idxs_srm_succ
    
    idxs_srm_fail = []
    for tgt_rec in tgt_mx[tgt_mx[:, 4] == 0]:
        tl_l, tl_r = tgt_rec[2], tgt_rec[3]
        idxs_srm_fail += list(np.where((idxs_srm_tl > tl_l) & (idxs_srm_tl < tl_r))[0])
    idxs_srm_fail_all[session] = idxs_srm_fail
        
    tsne_fits[session] = {}
    for perp in perplexities:
        tsne = TSNE(n_components=2, perplexity=perp, random_state=0)
        tsne_fit = tsne.fit_transform(combined_mx)
        tsne_fits[session][perp] = tsne_fit
        
    umap_fits[session] = {}
    for dist in umap_dists:
        umap_2d = UMAP(n_components=2, n_neighbors=30, min_dist=dist, random_state=0)
        umap_fit = umap_2d.fit_transform(combined_mx)
        umap_fits[session][dist] = umap_fit

    with h5py.File(moseq_class_file, 'w') as f:
        f.create_dataset('syl_ratio_mx', data=syl_ratio_mx)
        f.create_dataset('idxs_srm_tl', data=idxs_srm_tl)
        f.create_group('tSNE')
        for perp in perplexities:
            f['tSNE'].create_dataset(str(perp), data=tsne_fits[session][perp])
        f.create_group('UMAP')
        for dist in umap_dists:
            f['UMAP'].create_dataset(str(dist), data=umap_fits[session][dist])
        
    print(session)

/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/sobol

009265_hippoSIT_2023-03-09_20-03-08


/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/sobolev/projects/pySIT/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/sobol

009265_hippoSIT_2023-03-05_11-52-17


In [20]:
l_events = len(events)

combined_mx = np.column_stack([syl_ratio_mx[:l_events], w_mx[:l_events]])
combined_mx.shape

(9599, 14)